IMPORTS

In [10]:
import tensorflow as tf
from flask import Flask, request, jsonify
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
import numpy as np
import os

INSTACIAR SERVIDOR

In [11]:
app = Flask(__name__)

VERIFICAR SE O MODELO EXISTE

In [12]:
model_path = 'models/CNN_model.keras'
if not os.path.exists(model_path):
  raise FileExistsError(f"O arquivo {model_path} não foi encontrado")

CARREGAR MODELO

In [13]:
try:
  model = load_model(model_path)
  print("Modelo carregado com sucesso")
except Exception as e:
  print(f"Erro ao carregar modelo: {e}")

Modelo carregado com sucesso


PROCESSAR IMAGEM

In [14]:
def preprocess_image(img_path):
    img = image.load_image(img_path, target_size(224, 224))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = img_array / 255.0
    return img_array

CRIAR ROTA 

In [15]:
@app.route("/predict", methods=["POST"])
def predict():
    if "image" not in request.files:
        return jsonify({"error": "No image file in the request"}), 400

    img_file = request.files["image"]
    img_path = os.path.join("uploads", img_file.filename)
    img_file.save(img_path)

    img_array = preprocess_image(img_path)
    prediction = model.predict(img_array)

    response = {
        "predictions": {
            "COVID": f"{prediction[0][0] * 100:.2f}%",
            "Pneumonia": f"{prediction[0][1] * 100:.2f}%",
            "Normal": f"{prediction[0][2] * 100:.2f}%",
        }
    }

    return jsonify(response)

INICIALIZAR SERVIDOR

In [16]:
if __name__ == "__main__":
    if not os.path.exists("uploads"):
        os.makedirs("uploads")
    app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
